In [1]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft
import astropy.units as u
from astropy.visualization import quantity_support

In [2]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x23bc6b13b50>

In [3]:


pinhole_diameter = 44

# x_oversample = 8
#
# x = np.arange(64*x_oversample)
# x = x - x.shape[0]//2
#
#
# x,y = np.meshgrid(x,x)


x_oversample = 4

x = np.arange(128*2*x_oversample)
x = x - x.shape[0]//2


y = x.copy()

shp_x = x.shape[0]
x = x[...,None]
y = y[None,...]

x = np.broadcast_to(x,(shp_x,shp_x))
y = np.broadcast_to(y,(shp_x,shp_x))

r = np.sqrt(x*x + y*y)


pinhole = np.ones_like(r)
pinhole[r<pinhole_diameter/2 * x_oversample] = 0

fig, ax = plt.subplots(figsize=(4,4))
ax.pcolormesh(x/x_oversample,y/x_oversample, pinhole)
# ax.imshow(pinhole)


<IPython.core.display.Javascript object>

In [4]:
# wv = 10*u.AA
# f = 19*u.cm
#
# aperture = shp_x * u.micron
#
#
#
# f_number = f/aperture
#
# fresnel_zone = np.sqrt(wv*f).to(u.micron)
# print('First Fresnel Zone Diameter', fresnel_zone*2)
#
# oversample = 4
#
# dx = wv * f_number / oversample
# dx = dx.to(u.micron)
#
#|
# wfe = pinhole * 1e9 * 1j

def wfe2psf(wfe,oversample):
    shp = wfe.shape[0]
    waves = np.exp(2*np.pi*wfe*1j)

    big_waves = np.zeros((shp*oversample,shp*oversample),dtype='complex')
    big_waves[0:shp,0:shp] = waves

    psf = scipy.fft.fft2(big_waves,workers=-1)
    psf = np.abs(psf)**2
    psf /= psf.max()

    psf = np.roll(psf,(shp//2,shp//2),axis=[0,1])
    psf = psf[0:shp,0:shp]

    return psf

# print(psf.shape)
# fig,ax = plt.subplots()
# ax.plot(x[:,shp_x//2]*dx,psf[:,shp_x//2])

Add Fresnel Phase Factor

In [5]:
f = 19.5*u.cm
pixel_pitch = 7*u.micron/u.pixel

spatial_plate_scale = (pixel_pitch/f).to(1/u.pixel)*u.radian
spatial_plate_scale = spatial_plate_scale.to(u.arcsec/u.pixel)
print(spatial_plate_scale)


aperture = (shp_x / x_oversample) * u.micron
f_number = f/aperture
print(f_number.to(u.micron/u.micron))

7.404377660152177 arcsec / pix
761.71875


In [6]:



# fig,ax = plt.subplots()
# # ax.pcolormesh(x/x_oversample,y/x_oversample, tilt*0**pinhole)
# ax.imshow(np.real(tilt))


In [7]:
fig, ax = plt.subplots()


wvs = [5*u.AA, 10*u.AA, 25*u.AA, 50*u.AA]
ovs = [4,4,4,4]

# wvs = [25*u.AA]
# ovs = [4]
thetas = np.arange(5)*10 * u.arcsec
thetas = [0] * u.arcsec
# ovs = [1,1,1,1]

psfs = []
xs = []
for wv,oversample in zip(wvs,ovs):
    wv_micron = wv.to(u.micron).value
    for theta in thetas:
        # oversample=16

        fresnel_zone = np.sqrt(wv*f).to(u.micron)
        print('First Fresnel Zone Diameter', fresnel_zone*2)


        tilt = np.tan(theta).value * (x / (x_oversample*wv_micron)) + np.tan(0*u.arcsec).value * (y / (wv_micron*x_oversample)) + 0.j


        wfe = pinhole * 1e9 * 1j
        wfe += tilt



        dx = wv * f_number / oversample

        print(dx)
        dx = dx.to(u.micron)
        print(dx)
        dx = dx/pixel_pitch*spatial_plate_scale


        f_prime = f/np.cos(theta)
        print(f_prime)
        fresnel_wfe = 1/(2*wv*f_prime)*((x*x+y*y)/x_oversample**2) * u.micron**2 + 0.j
        fresnel_wfe = fresnel_wfe.value
        wfe_total = wfe+fresnel_wfe

        # psf = wfe2psf(wfe,oversample)
        fresnel_psf = wfe2psf(wfe_total,oversample)

        print(x.shape)
        print(fresnel_psf.shape)
        fig2, ax2 = plt.subplots()
        ax2.pcolormesh(x*dx,y*dx,fresnel_psf)
        ax2.set_xlim([-50*u.arcsec, 50*u.arcsec])
        ax2.set_ylim([-50*u.arcsec, 50*u.arcsec])

        x_1D = x[:, shp_x // 2] * dx
        psf_1D = fresnel_psf[:, shp_x // 2]
        ax.plot(x_1D, psf_1D, label=wv)
        psfs.append(psf_1D)
        xs.append(x_1D)

pin = pinhole_diameter*u.micron / pixel_pitch*spatial_plate_scale / 2
ax.vlines([-pin,pin],[0,0],[1,1],color = 'black', label='pinhole')

ax.set_xlim([-50*u.arcsec, 50*u.arcsec])
ax.legend()
ax.set_title('PSF for f ='+str(f)+' and d='+str(pinhole_diameter*u.micron))


    # ax.plot(x[:,shp_x//2]*dx,psf[:,shp_x//2])


<IPython.core.display.Javascript object>

First Fresnel Zone Diameter 19.7484176581315 micron
0.09521484375 Angstrom cm / micron
0.09521484375000003 micron
19.5 cm
(1024, 1024)
(1024, 1024)


<IPython.core.display.Javascript object>

First Fresnel Zone Diameter 27.92848008753789 micron
0.1904296875 Angstrom cm / micron
0.19042968750000006 micron
19.5 cm
(1024, 1024)
(1024, 1024)


<IPython.core.display.Javascript object>

First Fresnel Zone Diameter 44.15880433163924 micron
0.47607421875 Angstrom cm / micron
0.4760742187500001 micron
19.5 cm
(1024, 1024)
(1024, 1024)


<IPython.core.display.Javascript object>

First Fresnel Zone Diameter 62.449979983983994 micron
0.9521484375 Angstrom cm / micron
0.9521484375000002 micron
19.5 cm
(1024, 1024)
(1024, 1024)


<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'PSF for f =19.5 cm and d=44.0 micron')

In [8]:
import scipy.interpolate


new_x = np.linspace(-75,75,301) * u.arcsec

fig,ax = plt.subplots()
new_psfs = []
for psf, x  in zip(psfs,xs):
    func = scipy.interpolate.interp1d(x = x, y = psf)
    new_psf = func(new_x)
    new_psfs.append(new_psf)
    ax.plot(new_x,new_psf)

ax.vlines([-pin,pin],[0,0],[1,1],color = 'black', label='pinhole')

ax.set_xlim([-50*u.arcsec, 50*u.arcsec])
ax.legend()
ax.set_title('PSF for f ='+str(f)+' and d='+str(pinhole_diameter*u.micron))



<IPython.core.display.Javascript object>

ValueError: A value (-75.0) in x_new is below the interpolation range's minimum value (-51.5662015617741).

In [ ]:
sigma = 7 * u.arcsec
PV = 15 * u.arcsec
jitter_gaussian = np.exp(-new_x**2/(2*sigma**2))
jitter_rect = np.ones_like(new_x)
jitter_rect[new_x > PV/2] = 0
jitter_rect[new_x < -PV/2] = 0

fig,ax = plt.subplots()
ax.plot(new_x,jitter_gaussian)
ax.plot(new_x,jitter_rect.value)

In [ ]:
fig, ax = plt.subplots()
for psf in new_psfs:
    rect_psf = np.convolve(psf,jitter_rect,mode='same')
    rect_psf /= rect_psf.sum()
    ax.plot(new_x,rect_psf)
    print('FWHM = ' +str(2*np.abs(new_x[np.argmin(np.abs(rect_psf-rect_psf.max()*.5))])))



In [ ]:
fig, ax = plt.subplots()
for psf in new_psfs:
    gauss_psf = np.convolve(psf,jitter_gaussian,mode='same')
    gauss_psf /= gauss_psf.sum()
    ax.plot(new_x,gauss_psf)
    print('FWHM = ' +str(2*np.abs(new_x[np.argmin(np.abs(gauss_psf-gauss_psf.max()*.5))])))
